# Data preparation, exploration and sorting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Downloading packages and loading libraries
This section imports all the necessary libraries and modules required for the script to run. It includes libraries for data manipulation, visualization, machine learning models, image preprocessing, and model evaluation.

In [ ]:
! pip install tensorflow_addons
! pip install visualkeras

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from pathlib import Path
import tensorflow as tf
from random import randint
from tensorflow import keras
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from keras.utils import plot_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from distutils.dir_util import copy_tree, remove_tree
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, MaxPool2D, GlobalAveragePooling2D

# Installing the Kaggle API client and setting up the Kaggle configuration file

After running these commands, it is possible to use the Kaggle API client to interact with Kaggle datasets. `kaggle.json` file is created via Kaggle and uploaded manually to Colab.

In [ ]:
# Install the Kaggle API client package using pip
! pip install kaggle
# Create a directory named `.kaggle` in the user's home directory (`~`), which is where the Kaggle configuration file will be stored
! mkdir ~/.kaggle
# Copy the Kaggle API credentials file (`kaggle.json`) to the newly created `.kaggle` directory
! cp kaggle.json ~/.kaggle/
# Change the permissions of the `kaggle.json` file to make it readable and writable only by the owner (the user), ensuring that the API credentials are secure
! chmod 600 ~/.kaggle/kaggle.json


#Setting the random seed to the same value
This ensures reproducibility of results when using random number generation functions, which is especially important in machine learning tasks where randomness is involved, such as weight initialization, data shuffling, or dropout. By setting the random seed, random numbers generated during different runs of the script remain the same, providing consistent and comparable results.

In [ ]:
tf.random.set_seed(5638)
random.seed(5638)

#Downloading the curated dataset from Kaggle

This dataset is related to dementia and contains images categorized into three classes.

In [ ]:
! kaggle datasets download -d zofiaknapiska/adni-selected-processed-brain-axial-t1-mri-jpeg

#Unzipping and exploring the dataset

In [ ]:
! unzip '/content/adni-selected-processed-brain-axial-t1-mri-jpeg.zip'

#Preparing the data for training and testing

#Defining parameters for training a neural network model

Setting parameters and configurations for image processing and neural network training. These parameters help in standardizing the input images and controlling the training process, ensuring consistency and efficiency.

In [ ]:
# Define the number of epochs, which is the number of times the entire dataset will be passed forward and backward through the neural network during training
EPOCHS = 50

# Set the batch size, which determines the number of samples that will be propagated through the network at a time
BATCH_SIZE = 32

# Define the size to which the images will be resized in pixels
IMG_SIZE = 192

#Store the image size as a list with two elements, both set to 192
IMAGE_SIZE = [192, 192, 3]

# Define the dimensions to which the images will be resized as a tuple
DIM = (IMG_SIZE, IMG_SIZE)

# Visualizing the distribution of labels in the training set
The labels correspond to the different types of dementia in the training dataset. They provide insight into the balance or imbalance of classes, which is crucial for training a machine learning model effectively.

In [ ]:
# Path to the directory containing training images
train_dir = Path('/content/archive/train')

# Get filepaths and labels
filepaths = list(train_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
train_df = pd.concat([filepaths, labels], axis=1)

# Shuffle the DataFrame and reset index
train_df = train_df.sample(frac=1).reset_index(drop = True)

# Split training data into train and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['Label'], random_state=5638)

# Visualizing the distribution of labels in the testing set
The labels correspond to the different types of dementia in the testing dataset. They provide insight into the balance or imbalance of classes, which is crucial for testing a machine learning model effectively.

In [ ]:
# Path to the directory containing testing images
test_dir = Path('/content/archive/test')

# Get filepaths and labels
filepaths = list(test_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
test_df = pd.concat([filepaths, labels], axis=1)

# Shuffle the DataFrame and reset index
test_df = test_df.sample(frac=1).reset_index(drop = True)

In [ ]:
plt.figure(figsize=(8, 6), dpi=100)  # Set figure size and dpi
plt.subplots_adjust(left=0.15, right=0.9, top=0.9, bottom=0.15)  # Adjust padding

# Plot the grouped sizes as a horizontal bar plot
ax = train_df.groupby('Label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))

# Set the title of the x-axis
plt.xlabel('Counts')

# Set the title of the plot
plt.title('Training set - label distribution')

# Remove the spines on the top and right sides of the plot
plt.gca().spines[['top', 'right']].set_visible(False)

# Add text annotations for each bar to display the counts
for i in ax.patches:
    ax.text(i.get_width() + 0.1, i.get_y() + 0.1, str(round(i.get_width())), fontsize=10, color='black')

# Show the plot
plt.savefig('train_set_label_distribution.png', bbox_inches='tight')
plt.show()


plt.figure(figsize=(8, 6), dpi=100)  # Set figure size and dpi
plt.subplots_adjust(left=0.15, right=0.9, top=0.9, bottom=0.15)  # Adjust padding

# Plot the grouped sizes as a horizontal bar plot
cx = val_df.groupby('Label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))

# Set the title of the x-axis
plt.xlabel('Counts')

# Set the title of the plot
plt.title('Validation set - label distribution')

# Remove the spines on the top and right sides of the plot
plt.gca().spines[['top', 'right']].set_visible(False)

# Add text annotations for each bar to display the counts
for i in cx.patches:
    cx.text(i.get_width() + 0.1, i.get_y() + 0.1, str(round(i.get_width())), fontsize=10, color='black')

# Show the plot
plt.savefig('val_set_label_distribution.png', bbox_inches='tight')
plt.show()

plt.figure(figsize=(8, 6), dpi=100)  # Set figure size and dpi
plt.subplots_adjust(left=0.15, right=0.9, top=0.9, bottom=0.15)  # Adjust padding

# Plot the grouped sizes as a horizontal bar plot
bx = test_df.groupby('Label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))

# Set the title of the x-axis
plt.xlabel('Counts')

# Set the title of the plot
plt.title('Testing set - label distribution')

# Remove the spines on the top and right sides of the plot
plt.gca().spines[['top', 'right']].set_visible(False)

# Add text annotations for each bar to display the counts
for i in bx.patches:
    bx.text(i.get_width() + 0.1, i.get_y() + 0.1, str(round(i.get_width())), fontsize=10, color='black')

# Show the plot
plt.savefig('test_set_label_distribution.png', bbox_inches='tight')
plt.show()

#Utilizing `ImageDataGenerator` to prepare image data for training and testing
Setting up data pipelines for training and testing deep learning models using Keras. Automating data preprocessing tasks such as scaling and batching, making it easier to work with large datasets efficiently.

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.0)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(192,192),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    seed = 5638,
    shuffle=True
)

val_images = test_generator.flow_from_dataframe(
    dataframe=val_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(192, 192),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    seed=5638,
    shuffle=False
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(192, 192),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    seed = 5638,
    shuffle=False
)

CLASSES = list(test_images.class_indices.keys())
num_classes = len(CLASSES)

#Calculating and visualizing the distribution of classes within both the training and testing datasets

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Retrieve class indices mapping for both training and testing datasets
train_class_indices = train_images.class_indices
val_class_indices = val_images.class_indices
test_class_indices = test_images.class_indices

# Get the total number of classes present in the dataset
num_classes = len(train_class_indices)

# Initialize dictionaries to store counts of each class in the training and testing datasets
train_class_counts = {class_name: 0 for class_name in train_class_indices}
val_class_counts = {class_name: 0 for class_name in val_class_indices}
test_class_counts = {class_name: 0 for class_name in test_class_indices}

# Update counts for each class in the training set
for label in train_images.labels:
    class_name = CLASSES[label]
    train_class_counts[class_name] += 1

# Update counts for each class in the validation set
for label in val_images.labels:
    class_name = CLASSES[label]
    val_class_counts[class_name] += 1

# Update counts for each class in the testing set
for label in test_images.labels:
    class_name = CLASSES[label]
    test_class_counts[class_name] += 1

# Combine counts for both training and testing datasets
combined_class_counts = {}
for class_name in train_class_counts.keys():
    combined_class_counts[class_name] = [train_class_counts[class_name], val_class_counts[class_name], test_class_counts[class_name]]

# Plotting
plt.figure(figsize=(10, 6))
bar_width = 0.25
index = np.arange(num_classes)

# Plot bars for the training set
train_bars = plt.bar(index, [count[0] for count in combined_class_counts.values()], bar_width,
                     label='Train Set', color='b')

# Plot bars for the validation set
val_bars = plt.bar(index + bar_width, [count[1] for count in combined_class_counts.values()], bar_width,
                   label='Validation Set', color='g')

# Plot bars for the testing set
test_bars = plt.bar(index + 2 * bar_width, [count[2] for count in combined_class_counts.values()], bar_width,
                    label='Test Set', color='r')

# Set labels and title for the plot
plt.xlabel('Classes')
plt.ylabel('Counts')
plt.title('Class Distribution')
plt.xticks(index + bar_width, combined_class_counts.keys(), rotation=45, ha='right')
plt.legend()

# Adjust layout and display the plot
plt.tight_layout()
plt.savefig('class_label_distribution.png', bbox_inches='tight')
plt.show()

#Displaying 20 sample images from the dataset along with their corresponding labels

A visual representation of a subset of the dataset, allowing for a quick inspection of images and their associated labels.

In [ ]:
# Create a figure and a grid of subplots with 5 rows and 5 columns, totaling 20 subplots
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(10, 10),
                        subplot_kw={'xticks': [], 'yticks': []})

# Iterate over each subplot in the grid
for i, ax in enumerate(axes.flat):
    # Load and display the image corresponding to the filepath at index i in the DataFrame 'test_df'
    ax.imshow(plt.imread(test_df.Filepath[i]), cmap='gray')
    # Set the title of the subplot to the label of the image at index i in 'test_df'
    ax.set_title(test_df.Label[i])

# Adjust the layout to ensure proper spacing between subplots
plt.tight_layout()

# Display the plot containing the grid of images and labels
plt.savefig('dataset_sample.png', bbox_inches='tight')
plt.show()

#Working with the CNN

#Defining a convolutional neural network (CNN) model

In [ ]:
from sklearn.utils import class_weight

# Compute class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['Label']),
    y=train_df['Label']
)
class_weights = {i: weight for i, weight in enumerate(class_weights)}

In [ ]:
custom_model = Sequential([
    Input(shape=(IMAGE_SIZE)),
    Conv2D(16, 3, activation='relu', padding='same'),
    Conv2D(16, 3, activation='relu', padding='same'),
    MaxPool2D(),
    Conv2D(32, 3, activation='relu', padding='same'),
    Conv2D(32, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPool2D(),
    Conv2D(64, 3, activation='relu', padding='same'),
    Conv2D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPool2D(),
    Conv2D(128, 3, activation='relu', padding='same'),
    Conv2D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPool2D(),
    Conv2D(256, 3, activation='relu', padding='same'),
    Conv2D(256, 3, activation='relu', padding='same', name = 'last_conv_layer'),
    BatchNormalization(),
    MaxPool2D(),
    Flatten(),
    Dropout(0.2),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.7),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(3, activation='softmax')
], name = "custom_model")

In [ ]:
from contextlib import redirect_stdout
with open('modelsummary_model_custom.txt', 'w') as f:
    with redirect_stdout(f):
        custom_model.summary(show_trainable = True)

# Display model summary
custom_model.summary(show_trainable = True)

In [ ]:
import visualkeras

font = ImageFont.truetype('arial.ttf', 12)
visualkeras.layered_view(custom_model, to_file='model_custom_viskeras.png', legend=True)

In [ ]:
# Calling plot_model method and displaying a model structure summary
plot_model(custom_model, to_file='custom_model_structure.png', show_shapes=True, show_layer_names=True, rankdir='TB', expand_nested=False, dpi=100, show_layer_activations=True, show_trainable=True)

#Setting up the optimizer, defining evaluation metrics, and compiling the model for training



In [ ]:
# Define the Adam optimizer with a learning rate of 0.001 and assign it to the variable OPT
OPT = tf.keras.optimizers.Adam(learning_rate=0.0001)

# Define a list of evaluation metrics to be used during model training and validation
METRICS = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),  # Computes accuracy for multi-class classification
    tf.keras.metrics.AUC(name='auc'),                  # Computes area under the ROC curve for binary classification
    tfa.metrics.F1Score(num_classes=3)                 # Computes F1 score for multi-class classification with 3 classes
]

# Compile the model
custom_model.compile(optimizer=OPT, loss=tf.losses.CategoricalCrossentropy(), metrics=METRICS)

#Defining callbacks

Essential for monitoring the training process, preventing overfitting, and saving the best model weights.



In [ ]:
# Configure early stopping, halting training when the validation loss stops decreasing
earlystopping = EarlyStopping(
    monitor='val_loss',    # Monitor the validation loss during training
    mode='min',            # Stop training when the monitored quantity (val_loss) stops decreasing
    patience=7,            # Number of epochs with no improvement after which training will be stopped
    verbose=1              # Print messages about early stopping progress during training
)

# Define the filepath where the best model weights will be saved during training
filepath = './model_custom_best_weights.keras'

# Configure model checkpointing to save the best model weights based on validation loss
checkpoint = ModelCheckpoint(
    filepath,              # Filepath to save the best model weights
    monitor='val_loss',    # Monitor the validation loss during training
    mode='min',            # Save the weights when the monitored quantity (val_loss) achieves its minimum value
    save_best_only=True,   # Save only the best model weights (with the lowest validation loss)
    verbose=1              # Print messages about model checkpointing progress during training
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              min_lr=1e-6)

# Create a list of callbacks to be used during model training, including early stopping and model checkpointing
callback_list = [earlystopping, checkpoint, reduce_lr]

#Training and Validating a Model

Fitting the training data to a machine learning model and validating its performance using separate validation data. The code employs callbacks specified in `callback_list` to monitor and control the training process. The `epochs` parameter determines the number of times the entire training dataset is passed forward and backward through the model.


In [ ]:
# Train the model
custom_history = custom_model.fit(
    train_images,
    validation_data=val_images,
    callbacks=callback_list,
    epochs=EPOCHS,
    class_weight=class_weights  # Add class weights to handle class imbalance
)

#Monitoring Model Performance with Accuracy and Loss Plots

Generating four plots to monitor the performance of a custom machine learning model during training and validation.

In [ ]:
# Create a figure and axes for the accuracy plot
plt.figure(figsize=(8, 6))  # Set the size of the figure
plt.grid(True, linestyle=':', linewidth=0.5, color='gray')  # Add a delicate grid in the background
plt.plot(custom_history.history['acc'], linestyle='-', label='Training')  # Plot training accuracy without markers
plt.plot(custom_history.history['val_acc'], linestyle='--', label='Validation')  # Plot validation accuracy without markers
plt.title('Model Accuracy')  # Set title for the plot
plt.ylabel('Accuracy')  # Label y-axis
plt.xlabel('Epoch')  # Label x-axis
plt.xticks(range(0, len(custom_history.history['acc'])+1, max(1, len(custom_history.history['acc'])//10)))  # Dynamic x-axis based on the number of epochs

# Find and mark the best epoch for validation accuracy
best_epoch_val_acc = custom_history.history['val_acc'].index(max(custom_history.history['val_acc']))
plt.scatter(best_epoch_val_acc, max(custom_history.history['val_acc']), color='green', label=f'Best Epoch ({best_epoch_val_acc + 1})')

plt.legend(loc='lower right')  # Add legend to indicate which line represents train and validation
plt.tight_layout()  # Adjust layout to prevent overlapping of elements
plt.savefig('acc.png')
plt.show()  # Display the plot

# Create a figure and axes for the loss plot
plt.figure(figsize=(8, 6))  # Set the size of the figure
plt.grid(True, linestyle=':', linewidth=0.5, color='gray')  # Add a delicate grid in the background
plt.plot(custom_history.history['loss'], linestyle='-', label='Training')  # Plot training loss without markers
plt.plot(custom_history.history['val_loss'], linestyle='--', label='Validation')  # Plot validation loss without markers
plt.title('Model Loss')  # Set title for the plot
plt.ylabel('Loss')  # Label y-axis
plt.xlabel('Epoch')  # Label x-axis
plt.xticks(range(0, len(custom_history.history['loss'])+1, max(1, len(custom_history.history['loss'])//10)))  # Dynamic x-axis based on the number of epochs

# Find and mark the best epoch for validation loss
best_epoch_val_loss = custom_history.history['val_loss'].index(min(custom_history.history['val_loss']))
plt.scatter(best_epoch_val_loss, min(custom_history.history['val_loss']), color='green', label=f'Best Epoch ({best_epoch_val_loss + 1})')

plt.legend(loc='upper right')  # Add legend to indicate which line represents train and validation
plt.tight_layout()  # Adjust layout to prevent overlapping of elements
plt.savefig('loss.png')
plt.show()  # Display the plot

# Create a figure and axes for the AUC plot
plt.figure(figsize=(8, 6))  # Set the size of the figure
plt.grid(True, linestyle=':', linewidth=0.5, color='gray')  # Add a delicate grid in the background
plt.plot(custom_history.history['auc'], linestyle='-', label='Training')  # Plot training AUC without markers
plt.plot(custom_history.history['val_auc'], linestyle='--', label='Validation')  # Plot validation AUC without markers
plt.title('Model AUC')  # Set title for the plot
plt.ylabel('AUC')  # Label y-axis
plt.xlabel('Epoch')  # Label x-axis
plt.xticks(range(0, len(custom_history.history['auc'])+1, max(1, len(custom_history.history['auc'])//10)))  # Dynamic x-axis based on the number of epochs

# Find and mark the best epoch for validation AUC
best_epoch_val_auc = custom_history.history['val_auc'].index(max(custom_history.history['val_auc']))
plt.scatter(best_epoch_val_auc, max(custom_history.history['val_auc']), color='green', label=f'Best Epoch ({best_epoch_val_auc + 1})')

plt.legend(loc='lower right')  # Add legend to indicate which line represents train and validation
plt.tight_layout()  # Adjust layout to prevent overlapping of elements
plt.savefig('auc.png')
plt.show()  # Display the plot

# Get F1 scores from the history
f1_scores_train = custom_history.history['f1_score']
f1_scores_val = custom_history.history['val_f1_score']

# Create a figure and axes for the F1 score plot
plt.figure(figsize=(8, 6))  # Set the size of the figure
plt.grid(True, linestyle=':', linewidth=0.5, color='gray')  # Add a delicate grid in the background

# Plot training F1 score with a solid line
f1_scores_train_avg = np.mean(f1_scores_train, axis=1)
plt.plot(f1_scores_train_avg, linestyle='-', label='Training')

# Plot validation F1 score with a dashed line
f1_scores_val_avg = np.mean(f1_scores_val, axis=1)
plt.plot(f1_scores_val_avg, linestyle='--', label='Validation')

# Find and mark the best epoch for validation F1 score
best_epoch_val_f1 = np.argmax(f1_scores_val_avg)
plt.scatter(best_epoch_val_f1, f1_scores_val_avg[best_epoch_val_f1], color='green', zorder=5, label=f'Best Epoch ({best_epoch_val_f1 + 1})')

plt.title('Model F1 Score')  # Set title for the plot
plt.ylabel('F1 Score')  # Label y-axis
plt.xlabel('Epoch')  # Label x-axis
plt.xticks(range(0, len(f1_scores_train_avg) + 1, max(1, len(f1_scores_train_avg) // 10)))  # Dynamic x-axis based on the number of epochs

plt.legend(loc='lower right')  # Add legend to indicate which line represents train and validation
plt.tight_layout()  # Adjust layout to prevent overlapping of elements
plt.savefig('f1_score.png')
plt.show()  # Display the plot

#Evaluating Model Performance and Generating Classification Report

The performance of a custom machine learning model is evaluated using the test dataset.

First, the `evaluate` method is called on the model with the test images, providing evaluation scores. The accuracy of the model on the test data is then printed.

Next, the model predicts labels for the test images using the `predict` method. The predicted labels are then processed using a `roundoff` function, which rounds off each predicted label array by setting non-maximum values to 0 and the maximum value to 1.

After rounding off the predicted labels, the index of the maximum value along axis 1 is extracted to get the final predicted labels.

Finally, a classification report is generated using the `classification_report` function from scikit-learn, comparing the predicted labels against the true labels in the test dataset. The classification report provides metrics such as precision, recall, and F1-score for each class, aiding in the evaluation of the model's performance across different classes.

In [ ]:
from google.colab import files
files.download('/content/model_custom_best_weights.keras')

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

model_path = '/content/model_custom_best_weights.keras'
# Load the model
loaded_model = load_model(model_path)

# Evaluate the model on test data
test_scores = loaded_model.evaluate(test_images)
print("Testing Accuracy: %.2f%%" % (test_scores[1] * 100))

# Predict labels for test images
pred_labels = loaded_model.predict(test_images)

def roundoff(arr):
    """To round off according to the argmax of each predicted label array."""
    arr[np.argmax(arr)] = 1  # Set maximum value to 1
    arr[arr != 1] = 0  # Set non-maximum values to 0
    return arr

for i in range(len(pred_labels)):
    pred_labels[i] = roundoff(pred_labels[i])  # Round off the predicted labels

pred = np.argmax(pred_labels, axis=1)  # Get the index of the maximum value along axis 1

# Print classification report
print(classification_report(test_images.classes, pred, target_names=CLASSES, zero_division=1))

import sys
# Save the original standard output
original_stdout = sys.stdout

# Open a file in write mode to save the information
with open('loaded_model_evaluation_info.txt', 'w') as f:
    # Redirect the standard output to the file
    sys.stdout = f

    # Evaluate the model on test data
    test_scores = loaded_model.evaluate(test_images)
    print("Testing Accuracy: %.2f%%" % (test_scores[1] * 100))

    # Predict labels for test images
    pred_labels = loaded_model.predict(test_images)

    for i in range(len(pred_labels)):
        pred_labels[i] = roundoff(pred_labels[i])  # Round off the predicted labels

    pred = np.argmax(pred_labels, axis=1)  # Get the index of the maximum value along axis 1

    # Print classification report to the file
    print(classification_report(test_images.classes, pred, target_names=CLASSES, zero_division=1))

# Reset the standard output to the original value
sys.stdout = original_stdout

#Visualizing Classification Performance with Confusion Matrix

First, true and predicted labels are extracted from the test dataset and the model predictions, respectively. Then, the confusion matrix is computed using the `confusion_matrix` function from scikit-learn, which provides a summary of the model's predictions versus the actual labels.

In [ ]:
# Plot the confusion matrix to understand the classification in detail
test_ls, pred_ls = test_images.classes, pred  # Get true and predicted labels
conf_arr = confusion_matrix(test_ls, pred_ls)  # Compute confusion matrix

# Compute percentages
conf_arr_percent = conf_arr.astype('float') / conf_arr.sum(axis=1)[:, np.newaxis] * 100

plt.figure(figsize=(8, 6), dpi=100)  # Set figure size and dpi
plt.subplots_adjust(left=0.15, right=0.9, top=0.9, bottom=0.15)  # Adjust padding

# Plot heatmap with annotations
ax = sns.heatmap(conf_arr_percent, cmap='Blues', annot=True, fmt='.1f', cbar=True, xticklabels=CLASSES, yticklabels=CLASSES)

for t in ax.texts:
    t.set_text(t.get_text() + ' %')

plt.title('Alzheimer\'s Disease Diagnosis')  # Set title for the plot
plt.xlabel('Predicted Labels')  # Set label for x-axis
plt.ylabel('True Labels')  # Set label for y-axis
plt.savefig('confusion_matrix_loaded_model.png', bbox_inches='tight')  # Save the plot with tight bounding box
plt.show()  # Display the plot

#Analyzing Model Performance with ROC Curve and Thresholds

Analysis of model performance using Receiver Operating Characteristic (ROC) curves by incorporating different classification thresholds.

After converting true labels into a binary format, ROC curves and areas under the curve (AUC) are computed for each class. Additionally, the true positive rate (TPR) and false positive rate (FPR) are calculated for various classification thresholds.

The micro-average ROC curve and AUC are derived by aggregating TPR and FPR across all classes, while the macro-average ROC curve and AUC represent the average performance across all classes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.preprocessing import label_binarize

# Evaluate the model on test data
test_scores = loaded_model.evaluate(test_images)
print("Testing Accuracy: %.2f%%" % (test_scores[1] * 100))

# Predict probabilities for test images
pred_labels = loaded_model.predict(test_images)

# Convert labels to binary format where each class is represented by a binary vector
y_true = label_binarize(test_images.classes, classes=range(len(CLASSES)))  # CLASSES is the list of class names
n_classes = y_true.shape[1]

# Compute ROC curve and ROC area for each class
fpr = dict()  # False Positive Rate
tpr = dict()  # True Positive Rate
roc_auc = dict()  # Area under ROC curve

# Thresholds for which to compute ROC
thresholds = np.linspace(0, 1, 100)  # 100 thresholds between 0 and 1
tpr_values = np.zeros((n_classes, len(thresholds)))
fpr_values = np.zeros((n_classes, len(thresholds)))

# Iterate over each class to compute ROC curve and area
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true[:, i], pred_labels[:, i])  # Compute FPR, TPR, thresholds
    roc_auc[i] = auc(fpr[i], tpr[i])  # Compute area under ROC curve

    # Compute TPR and FPR for different thresholds
    for j, threshold in enumerate(thresholds):
        tp = np.sum((pred_labels[:, i] >= threshold) & (y_true[:, i] == 1))
        fp = np.sum((pred_labels[:, i] >= threshold) & (y_true[:, i] == 0))
        tn = np.sum((pred_labels[:, i] < threshold) & (y_true[:, i] == 0))
        fn = np.sum((pred_labels[:, i] < threshold) & (y_true[:, i] == 1))

        tpr_values[i, j] = tp / (tp + fn) if (tp + fn) > 0 else 0
        fpr_values[i, j] = fp / (fp + tn) if (fp + tn) > 0 else 0

# Compute micro-average ROC curve and ROC area
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))  # Obtain all unique FPR values
mean_tpr = np.zeros_like(all_fpr)  # Initialize mean TPR
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])  # Interpolate TPR values
mean_tpr /= n_classes  # Calculate mean TPR
fpr["micro"] = all_fpr
tpr["micro"] = mean_tpr
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])  # Compute area under micro-average ROC curve

# Compute macro-average ROC curve and ROC area
roc_auc_macro = auc(fpr["micro"], tpr["micro"])  # Compute area under macro-average ROC curve

# Plot ROC curve for each class
lw = 2  # Adjust line width if needed
plt.figure(figsize=(8, 6))

colors = ['aqua', 'darkorange', 'cornflowerblue']  # Adjust colors if needed
for i, color in zip(range(n_classes), colors):
    plt.step(fpr_values[i], tpr_values[i], color=color, lw=lw, where='post',
             label='ROC curve of class {0} (area = {1:0.3f})'.format(CLASSES[i], roc_auc[i]))

# Plot micro-average ROC curve
plt.plot(fpr["micro"], tpr["micro"], color='deeppink', linestyle=':', linewidth=4,
         label='Micro-average ROC curve (area = {0:0.3f})'.format(roc_auc["micro"]))

# Plot macro-average ROC curve
plt.plot(fpr["micro"], tpr["micro"], color='navy', linestyle='--', linewidth=4,
         label='Macro-average ROC curve (area = {0:0.3f})'.format(roc_auc_macro))

# Plot points for different thresholds
for i, color in zip(range(n_classes), colors):
    for j, threshold in enumerate(thresholds[::10]):
        plt.scatter(fpr_values[i, j*10], tpr_values[i, j*10], color=color, s=10)

# Add random guess line
plt.plot([0, 1], [0, 1], color='gray', lw=lw, linestyle='--')

# Set plot properties
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve with Thresholds')
plt.legend(loc="lower right")
plt.grid(True, linestyle=':', linewidth=0.5, color='gray')  # Add a delicate grid in the background
# Save the figure as a .png file
plt.savefig('roc_curve_loaded_model_new.png')
plt.show()

# Print classification report
pred = np.argmax(pred_labels, axis=1)  # Get the index of the maximum value along the axis 1
print(classification_report(test_images.classes, pred, target_names=CLASSES))

# # Save the evaluation and classification report to a file
import sys

# # Save the original standard output
original_stdout = sys.stdout

# # Open a file in write mode to save the information
with open('loaded_model_evaluation_info.txt', 'w') as f:
  # Redirect the standard output to the file
  sys.stdout = f
  # Evaluate the model on test data
  test_scores = loaded_model.evaluate(test_images)
  print("Testing Accuracy: %.2f%%" % (test_scores[1] * 100))  # Print the testing accuracy

  # Print classification report to the file
  print(classification_report(test_images.classes, pred, target_names=CLASSES))

  # Reset the standard output to the original value
  sys.stdout = original_stdout

#Misclassifications for each class

In [ ]:
# Initialize misclassifications for each class
misclassifications = np.zeros(len(CLASSES), dtype=int)

# Initialize total instances for each class
total_instances = np.zeros(len(CLASSES), dtype=int)

# Iterate over each instance in the test set
for true_label, pred_label in zip(test_ls, pred_ls):
    # Increment total instances count for the true label's class
    total_instances[true_label] += 1
    # If the true label and predicted label don't match, increment misclassification count for the true label's class
    if true_label != pred_label:
        misclassifications[true_label] += 1

# Calculate misclassifications as percentages
misclassifications_percentage = (misclassifications / total_instances) * 100


plt.figure(figsize=(8, 6))  # Set figure size
plt.subplots_adjust(left=0.15, right=0.9, top=0.9, bottom=0.15)  # Adjust padding

# Plot the bar chart
bars = plt.bar(CLASSES, misclassifications_percentage, color='skyblue')
plt.title('Misclassifications on Test Set by Class')
plt.xlabel('Class')
plt.ylabel('Misclassifications Percentage')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle=':', linewidth=0.5, color='gray')

# Annotate each bar with its percentage value
for bar, percentage in zip(bars, misclassifications_percentage):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{percentage:.2f}%', ha='center', va='bottom')

plt.savefig('missclass_loaded_model.png', bbox_inches='tight')  # Save the plot with tight bounding box
plt.show()  # Display the plot

#Printing Additional Classification Metrics

1. **Balanced Accuracy Score (BAS)** - a metric that provides a balanced measure of classification performance, particularly useful for imbalanced datasets. It considers both the sensitivity (true positive rate) and specificity (true negative rate) of the model's predictions.

2. **Matthew's Correlation Coefficient (MCC)** - a metric that measures the quality of binary classifications, taking into account both true and false positives and negatives. It ranges from -1 to 1, where 1 indicates perfect predictions, 0 indicates random predictions, and -1 indicates complete disagreement between predictions and true labels.

In [ ]:
# Open a text file in write mode
with open('classification_metrics_loaded_model.txt', 'w') as f:
    # Write Balanced Accuracy Score (BAS) to the file
    f.write("Balanced Accuracy Score: {} %\n".format(round(BAS(test_ls, pred_ls) * 100, 2)))

    # Write Matthew's Correlation Coefficient (MCC) to the file
    f.write("Matthew's Correlation Coefficient: {} %\n".format(round(MCC(test_ls, pred_ls) * 100, 2)))

# Print a message indicating that the metrics have been saved
print("Classification metrics have been saved to 'classification_metrics_loaded_model.txt'.")

# Printing some other classification metrics

# Compute BAS and format the output as a percentage rounded to two decimal places.
print("Balanced Accuracy Score: {} %".format(round(BAS(test_ls, pred_ls) * 100, 2)))

# Compute MCC and format the output as a percentage rounded to two decimal places.
print("Matthew's Correlation Coefficient: {} %".format(round(MCC(test_ls, pred_ls) * 100, 2)))

#Saving Custom CNN Model

Saving a custom Convolutional Neural Network (CNN) model along with its performance score in the filename. The model is saved in the Keras HDF5 format.

In [ ]:
# Save the custom CNN model
custom_model.save('custom_cnn.keras')

#Generating and Visualizing Gradient-weighted Class Activation Mapping (Grad-CAM)

Functions to generate and visualize Gradient-weighted Class Activation Mapping (Grad-CAM) for interpreting the predictions made by a convolutional neural network (CNN) model.

1. **`make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None)`:** This function computes the Grad-CAM heatmap for a given input image array, CNN model, and the name of the last convolutional layer. It calculates the gradient of the top predicted class with respect to the activations of the last convolutional layer, and then generates the heatmap by weighting the importance of each feature map channel. The heatmap is normalized between 0 and 1.

2. **`save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4)`:** This function loads the image, rescales the heatmap, and colorizes it using the jet colormap. It then superimposes the heatmap on the original image with a specified transparency level (`alpha`) and saves the resulting image. Finally, it returns the path of the saved image.

This code enables the visualization of which parts of an input image contribute most to the model's prediction, aiding in model interpretation and understanding.

In [ ]:
import matplotlib.cm as cm

def get_img_array(img_path, size):
    # Load the image from the given path and resize it to the specified size
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    # Convert the image to a numpy array
    array = tf.keras.preprocessing.image.img_to_array(img)
    # Add a batch dimension to the array
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Create a new model that maps the input image to the activations of the last convolutional layer as well as the output predictions
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])

    # Compute the gradient of the top predicted class for the input image with respect to the activations of the last convolutional layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Compute the gradient of the output neuron with respect to the output feature map of the last convolutional layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Compute the mean intensity of the gradient over each feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel in the feature map array by its importance and sum all channels to obtain the heatmap
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize the heatmap between 0 and 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.75):
    # Load the original image
    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = plt.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Return the path of the saved image
    return cam_path

# Define the size of the images
img_size = IMAGE_SIZE

# Remove the softmax activation from the last layer of the model
custom_model.layers[-1].activation = None

# Name of the last convolutional layer
last_conv_layer_name = "last_conv_layer"

#Making Predictions and Mapping Labels

Performing predictions on the test images using a custom CNN model. After obtaining the predicted labels, it rounds off each label array to assign a binary classification. Then, it maps the numerical label indices to their corresponding class names.

In [ ]:
# Perform predictions on the test images using the custom model
pred_labels = custom_model.predict(test_images)

def roundoff(arr):
    """Round off each array according to the index of its maximum value."""
    # Set all elements to 0 except for the maximum value, which is set to 1
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

# Iterate over each predicted label array and round off the values
for labels in pred_labels:
    labels = roundoff(labels)

# Obtain the index of the maximum value in each predicted label array
pred = np.argmax(pred_labels,axis=1)

# Map the numerical label indices to their respective class names
labels = (test_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

#Visualizing Neural Network Attention with Grad-CAM Heatmaps

Generating and displaying Grad-CAM heatmaps superimposed on original images to visualize the parts of the pictures that a neural network focuses on when classifying them. Grad-CAM highlights the regions of an image that are most influential in the model's decision-making process.

In [ ]:
import matplotlib.pyplot as plt

# Define the function to display images and Grad-CAM heatmaps neatly
def display_images_with_heatmaps(test_df, pred, img_size, custom_model, last_conv_layer_name):
    # Create a grid of subplots to display images and their heatmaps
    fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(12, 12), subplot_kw={'xticks': [], 'yticks': []})

    # Iterate over each subplot and corresponding image
    for i, ax in enumerate(axes.flat):
        # Get the file path of the image
        img_path = test_df.Filepath.iloc[i+164]
        # Load the image and convert it to an array
        img_array = get_img_array(img_path, size=img_size)
        # Generate the Grad-CAM heatmap for the image
        heatmap = make_gradcam_heatmap(img_array, custom_model, last_conv_layer_name)
        # Save and display the Grad-CAM heatmap superimposed on the original image
        cam_path = save_and_display_gradcam(img_path, heatmap)
        # Display the image with its Grad-CAM heatmap as a subplot
        img = plt.imread(cam_path)
        ax.imshow(img, cmap='jet')

        # Display ground truth, predicted label, and percentage
        ground_truth = test_df.Label.iloc[i]
        predicted_label = pred[i]

        # Set title with smaller font size for neatness
        ax.set_title(f"Ground truth: {ground_truth}\nPredicted label: {predicted_label}", fontsize=8)

    # Adjust layout and add colorbar spanning full length
    plt.tight_layout()
    cbar_ax = fig.add_axes([1, 0.1, 0.02, 0.8])  # Define colorbar axis (moved further to the right)
    plt.colorbar(ax.images[0], cax=cbar_ax)  # Add colorbar

    # Save and display the plot
    plt.savefig("grad_cam.jpg", bbox_inches='tight')
    plt.show()

# Call the function to display images with heatmaps
display_images_with_heatmaps(test_df, pred, img_size, custom_model, last_conv_layer_name)
